In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Input, Dropout
from keras.layers import Conv1D, MaxPooling1D, Conv2D, MaxPool2D
from keras.callbacks import LearningRateScheduler, EarlyStopping, TensorBoard
from keras.regularizers import l2

/home/ajuric/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load data

In [2]:
X_validate, y_validate = np.load('./dataset-n20-X-validate.npy'), np.load('./dataset-n20-y-validate.npy')
print(X_validate.shape)
print(y_validate.shape)

(3109078, 41, 5)
(3109078, 6)


## Model definition

In [3]:
def lr_schedule(epoch, lr):
    if epoch > 50:
        if epoch % 10 == 0:
            return lr * 0.95
    return lr

lr_callback = LearningRateScheduler(lr_schedule)
callbacks = [lr_callback, 
             EarlyStopping(monitor='loss', patience=3),]
#              TensorBoard(log_dir=tensorboard_output_dir, write_images=True, write_grads=True, histogram_freq=5, batch_size=10000)]

input_shape = X_validate.shape[1:]
num_output_classes = y_validate.shape[1]

input_layer = Input(shape=input_shape)

conv_1 = Conv1D(filters=16, kernel_size=4, padding='same', activation='selu', kernel_regularizer=l2(1e-3))(input_layer)
pool_1 = MaxPooling1D(pool_size=(5), strides=1)(conv_1)

conv_2 = Conv1D(filters=32, kernel_size=4, padding='same', activation='selu', kernel_regularizer=l2(1e-3))(pool_1)
pool_2 = MaxPooling1D(pool_size=(4), strides=1)(conv_2)

conv_3 = Conv1D(filters=48, kernel_size=4, padding='same', activation='selu', kernel_regularizer=l2(1e-3))(pool_2)
pool_3 = MaxPooling1D(pool_size=(3), strides=1)(conv_3)

flatten = Flatten()(pool_3)

dn_1 = Dense(336, activation='selu')(flatten)
drop = Dropout(0.5)(dn_1)

predictions = Dense(num_output_classes, activation='softmax')(drop)

model = Model(input_layer, predictions)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

batch_size = 10000
epochs = 150

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 41, 5)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 41, 16)            336       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 37, 16)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 37, 32)            2080      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 34, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 34, 48)            6192      
___________________________________________________________

In [4]:
model.fit(X_validate, y_validate, batch_size=batch_size, epochs=epochs, validation_data=(X_validate, y_validate), callbacks=callbacks)
model.save(model_save_path)

Train on 3109078 samples, validate on 3109078 samples
Epoch 1/150
3109078/3109078 [==============================] - 66s 21us/step - loss: 8.2841 - acc: 0.4533 - val_loss: 1.3858 - val_acc: 0.5007
Epoch 2/150
3109078/3109078 [==============================] - 65s 21us/step - loss: 1.0567 - acc: 0.6160 - val_loss: 0.5661 - val_acc: 0.7961
Epoch 3/150
3109078/3109078 [==============================] - 66s 21us/step - loss: 0.4760 - acc: 0.8390 - val_loss: 0.3169 - val_acc: 0.9091
Epoch 4/150
 620000/3109078 [====>.........................] - ETA: 37s - loss: 0.3486 - acc: 0.8958

KeyboardInterrupt: 